<a href="https://colab.research.google.com/github/robotics-upo/rva-course-material/blob/master/deeplearningbasics/deploying_imageclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

In this part, we will see how to use a model already trained in our code with OpenCV.

We will use it to detect objects from the CIFAR categories in images.

# Loading data and models

Let's load the model we trained in the previous session from file. We use in the virtual machine the Google Drive, which is mounted.

You can load the models and files from your local folders if you are not using Colab



In [ ]:
#Numpy module
import numpy as np

#Import OpenCV
import cv2
#Import tensorflow
import tensorflow as tf

#We can use OpenCV in Colab, but not its functions for creating plots
#We use matplotlib for generating plots
from matplotlib import pyplot as plt
from matplotlib import cm

#We use the library scikit to read images from url 
#In OpenCV, the function to read from file is cv2.imread
from skimage import io


In [ ]:
#Mount the drive
from google.colab import drive
drive.mount('/content/drive')

I am assuming you have a folder called **colabfiles** in your Google Drive **root** folder. 

Within the folder, we have the model **classif.h5**.

If the folders and files are called differently, change the paths and names.


In [ ]:
#Let's load the CNN model using the Keras API

model = tf.keras.models.load_model('/content/drive/My Drive/colabfiles/classif.h5')
print(model.summary())

Load the image and show it. Matplotlib assumes that images are RGB, and OpenCV stores them as BGR

In [ ]:

#Unmount the drive if we are not using it more
drive.flush_and_unmount()

#Let's load an image
imrgb = io.imread('https://robotics.upo.es/~lmercab/rva/test.jpg')

#We can use OpenCV in Colab, but not its function imshow
#We use matplotlib instead
from matplotlib import pyplot as plt

plt.imshow(imrgb, cmap=plt.cm.binary)

# Using our CNN for prediction

We can process patches through our network. Recall our network receives as inputs 32x32 images (normalized between 0 and 1) and outputs probabilities for each of the 10 classes of CIFAR-10

In [ ]:
import numpy as np

cifar10_labels = ['airplane', 'automobile', 'bird','cat','deer','dog','frog'
'horse','ship', 'truck']

#32x32 image patch, normalized
im_patch = imrgb[150:150+32,100:100+32]/255.0
plt.imshow(im_patch, cmap=plt.cm.binary)

#The network expect as inputs sets of images. We have to expand the dimension of
#the image
im_patch_final = np.expand_dims(im_patch, 0)

prediction = model.predict(im_patch_final)
print(prediction)

print("Predicción del modelo: ", cifar10_labels[np.argmax(prediction[0])] )


We can use our trained network to look for cars in the image. Also, we want to detect multiple cars, not only one.

The idea is to cover the whole image, extracting patches of 32x32 and taking those ones in which the network predicts and automobile

In [ ]:
#Loop over the image looking for cars
#We extract 32x32 patches each time
im_result = imrgb.copy()
for r in range(0,imrgb.shape[0] - 32, 32):
  for c in range(0,imrgb.shape[1] - 32, 32):
    im_patch_final = np.expand_dims(imrgb[r:r+32,c:c+32], 0)/255.0
    prediction = model.predict(im_patch_final)
    #Draw a rectangle on the original image if the probability of car is over 0.5
    if(np.argmax(prediction[0]) == 1 and prediction[0][1]>0.5):
      upper_left = (c, r)
      bottom_right = (c + 32, r+32)
      cv2.rectangle(im_result,upper_left, bottom_right, (255,0,0), 2)
     
plt.imshow(im_result, cmap=plt.cm.binary)

# Searching at different scales

The main problem is that our network expects cars as 32x32 patches. Of course, in our image there are cars larger than that. 

We need to search for cars in multiple scales in the image. For that, we can use image pyramids.

https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_pyramids/py_pyramids.html


In [ ]:
#Create an image pyramid

#Dowsample image by 2
imrgb_2 = cv2.pyrDown(imrgb)
plt.imshow(imrgb_2, cmap=plt.cm.binary)


In [ ]:
#Downsample the former by 2 (so the original by 4)
imrgb_4 = cv2.pyrDown(imrgb_2)
plt.imshow(imrgb_4, cmap=plt.cm.binary)


In [ ]:
#Search in downsampled images

#im_result_4 = imrgb_4.copy()
for r in range(0,imrgb_4.shape[0] - 32, 8):
  for c in range(0,imrgb_4.shape[1] - 32, 8):
    im_patch_final = np.expand_dims(imrgb_4[r:r+32,c:c+32], 0)/255.0
    prediction = model.predict(im_patch_final)
    #Draw a rectangle on the original image if the probability of car is over 0.5
    if(np.argmax(prediction[0]) == 1 and prediction[0][1]>0.5):
      upper_left = (c*4, r*4)
      bottom_right = (c*4 + 32*4, r*4+32*4)
      cv2.rectangle(im_result,upper_left, bottom_right, (0,255,0), 2)

for r in range(0,imrgb_2.shape[0] - 32, 8):
  for c in range(0,imrgb_2.shape[1] - 32, 8):
    im_patch_final = np.expand_dims(imrgb_2[r:r+32,c:c+32], 0)/255.0
    prediction = model.predict(im_patch_final)
    #Draw a rectangle on the original image if the probability of car is over 0.5
    if(np.argmax(prediction[0]) == 1 and prediction[0][1]>0.5):
      upper_left = (c*2, r*2)
      bottom_right = (c*2 + 32*2, r*2+32*2)
      cv2.rectangle(im_result,upper_left, bottom_right, (0,0,255), 2)
     
plt.imshow(im_result, cmap=plt.cm.binary)

#Using pre-trained models

In [ ]:
import tensorflow_hub as hub

classifier_model ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"

IMAGE_SHAPE = (224, 224)

classifier = tf.keras.Sequential([
    hub.KerasLayer(classifier_model, input_shape=IMAGE_SHAPE+(3,))
])

print(classifier.summary)

labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())


In [ ]:
im_input = io.imread('https://robotics.upo.es/~lmercab/rva/bookshelf.jpg')

im_input = im_input/255.0
print(im_input.shape)
plt.figure(figsize=(10,10))
plt.imshow(im_input)

In [ ]:
res = tf.image.resize(im_input, IMAGE_SHAPE)

print(res.shape)

result = classifier.predict(res[np.newaxis, ...])
print(result.shape)

predicted_class = np.argmax(result[0], axis=-1)
print(predicted_class)


In [ ]:
plt.figure()  
plt.imshow(res)
plt.axis('off')
predicted_class_name = imagenet_labels[predicted_class]
_ = plt.title("Prediction: " + predicted_class_name.title())

In [ ]:
from tensorflow.keras.utils import to_categorical

#Load train dataset for CIFAR10
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

from matplotlib import pyplot as plt
plt.imshow(x_test[100], cmap=plt.cm.binary)

#Normalize data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)


In [ ]:
model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4", output_shape=[1280],
                   trainable=False),  # Can be True.
    tf.keras.layers.Dense(10, activation='softmax')
])

model.build([None, 224, 224, 3])  # Batch input shape.

print(model.summary())

In [ ]:
x_resized = tf.image.resize(x_train[0:10000,:,:,:], IMAGE_SHAPE)

print(x_resized.shape)
print(y_train.shape)

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer="sgd",
              metrics = ['accuracy'])

_ = model.fit(x_resized, y_train[0:10000,:], epochs=10, verbose = 1)

In [ ]:
x_resized = tf.image.resize(x_test[2000:3000,:,:,:], IMAGE_SHAPE)

_ , test_acc = model.evaluate(x_resized, y_test[2000:3000,:])

print('Accuracy:', test_acc*100)